### Get the data in a format so we can call the API to translate it

In [25]:
import json

In [26]:
data_path = '../data/2023-04-12_oasst_all.trees.jsonl'
with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

### Translate the data

In [ ]:
from deep_translator import (GoogleTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeeplTranslator,
                             QcriTranslator,
                             single_detection,
                             batch_detection)

In [20]:
translators = [GoogleTranslator, MicrosoftTranslator, PonsTranslator, LingueeTranslator, MyMemoryTranslator, YandexTranslator, PapagoTranslator, DeeplTranslator, QcriTranslator]
GoogleTranslator().get_supported_languages()
"slovenian" in GoogleTranslator().get_supported_languages()

True

In [21]:
translated = GoogleTranslator(source='auto', target='sl').translate("keep it up, you are awesome")
translated

'kar tako naprej, super si'